In [2]:
import numpy as np 
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split


In [3]:
import re 
import nltk
from nltk.util import pr
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword = set()
stopword = set(stopwords.words("english"))

In [4]:
df = pd.read_csv("twitter_data.csv")
df.head

<bound method NDFrame.head of        Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0               0      3            0                   0        3      2   
1               1      3            0                   3        0      1   
2               2      3            0                   3        0      1   
3               3      3            0                   2        1      1   
4               4      6            0                   6        0      1   
...           ...    ...          ...                 ...      ...    ...   
24778       25291      3            0                   2        1      1   
24779       25292      3            0                   1        2      2   
24780       25294      3            0                   3        0      1   
24781       25295      6            0                   6        0      1   
24782       25296      3            0                   0        3      2   

                                             

In [5]:
df['labels'] = df['class'].map({0:"hate_speech detected",1:"offencive speech detected",2:"no hate speech and offencive speech detected"})
df.head

<bound method NDFrame.head of        Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0               0      3            0                   0        3      2   
1               1      3            0                   3        0      1   
2               2      3            0                   3        0      1   
3               3      3            0                   2        1      1   
4               4      6            0                   6        0      1   
...           ...    ...          ...                 ...      ...    ...   
24778       25291      3            0                   2        1      1   
24779       25292      3            0                   1        2      2   
24780       25294      3            0                   3        0      1   
24781       25295      6            0                   6        0      1   
24782       25296      3            0                   0        3      2   

                                             

In [6]:
df = df[['tweet','labels']]
df.head

<bound method NDFrame.head of                                                    tweet  \
0      !!! RT @mayasolovely: As a woman you shouldn't...   
1      !!!!! RT @mleew17: boy dats cold...tyga dwn ba...   
2      !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...   
3      !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...   
4      !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...   
...                                                  ...   
24778  you's a muthaf***in lie &#8220;@LifeAsKing: @2...   
24779  you've gone and broke the wrong heart baby, an...   
24780  young buck wanna eat!!.. dat nigguh like I ain...   
24781              youu got wild bitches tellin you lies   
24782  ~~Ruffled | Ntac Eileen Dahlia - Beautiful col...   

                                             labels  
0      no hate speech and offencive speech detected  
1                         offencive speech detected  
2                         offencive speech detected  
3                         offenci

In [7]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]','', text)
    text = re.sub('htpps?://\s+|www\.\S+','', text)
    text = re.sub('<.*?>+','',text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '',text)
    text = re.sub('\w*\d\W*','',text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text
df["tweet"] = df["tweet"].apply(clean)
print(df.head())   

                                               tweet  \
0   rt mayasolov woman shouldnt complain clean ho...   
1   rt boy dat coldtyga dwn bad cuffin dat hoe st...   
2   rt urkindofbrand dawg rt life ever fuck bitch...   
3             rt cganderson vivabas look like tranni   
4   rt shenikarobert shit hear might true might f...   

                                         labels  
0  no hate speech and offencive speech detected  
1                     offencive speech detected  
2                     offencive speech detected  
3                     offencive speech detected  
4                     offencive speech detected  


C:\Users\Kaustubh\AppData\Local\Temp\ipykernel_18412\2242842598.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["tweet"] = df["tweet"].apply(clean)


In [8]:
x = np.array(df["tweet"])
y = np.array(df["labels"])


cv = CountVectorizer()
x = cv.fit_transform(x)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3,random_state=0)


In [9]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier()
clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [10]:
from sklearn.metrics import accuracy_score
y_pred = clf.predict(x_test)

In [11]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8661735036987223


In [17]:
x1 =df["tweet"]
y1 = df["labels"]
x1_train, x1_test, y1_train, y1_test = train_test_split(x1,y1,test_size=0.3,random_state=2)
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
te = Pipeline(steps=[('cv',CountVectorizer()),('lr',LogisticRegression())])
te.fit(x1_train,y1_train)

c:\Python311\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('cv', CountVectorizer()), ('lr', LogisticRegression())])

In [18]:
y1_pred=te.predict(x1_test)
accuracy1 = accuracy_score(y1_test, y1_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8661735036987223


In [20]:
import pickle
with open('clf.pkl', 'wb') as handle:
    pickle.dump(clf, handle, protocol=pickle.HIGHEST_PROTOCOL)